# Week 3. Problem set

## Evgeny Bobkunov SD-03 e.bobkunov@innopolis.university

### 1. Implement the following functions in Racket using higher-order functions (`apply`, `map`, `andmap`, `ormap`, `filter`, `foldl`) and **without explicit recursion**.

#### (a) Find the most frequent symbol in a list (return the first one when ambiguous):


```racket
(most-frequent '(h e l l o w o r l d)) ; ==> 'l
```

In [54]:
(define (most-frequent lst)
  (define freq-alist
    (foldl (lambda (sym acc)
             (if (assoc sym acc)
                 (let* ((current-count (cdr (assoc sym acc))))
                   (map (lambda (pair)
                          (if (eq? (car pair) sym)
                              (cons sym (+ current-count 1))
                              pair))
                        acc))
                 (cons (cons sym 1) acc)))
           '()
           lst))

  (define (compare-freq pair1 pair2)
    (if (> (cdr pair1) (cdr pair2)) pair1 pair2))

  (car (foldl compare-freq (car freq-alist) (cdr freq-alist))))


(most-frequent '(h e l l o w o r l d))


'l

#### (b) Annotate each symbol in a list with its occurrence in the list:

```
(annotate-occurrence '(h e l l o w o r l d))
; ==> '((h 1) (e 1) (l 1) (l 2) (o 1) (w 1) (o 2) (r 1) (l 3) (d 1))
```

In [57]:
(define (annotate-occurrence lst)
  (define (update-freq sym acc)
    (let* ((occurrences (assoc sym (car acc)))
           (new-count (if occurrences
                          (+ 1 (cdr occurrences))
                          1))
           (new-acc (cons (cons sym new-count) (car acc))))
      (cons new-acc (cons (list sym new-count) (cdr acc)))))

  (cdr (foldr update-freq (cons '() '()) lst)))

(annotate-occurrence '(h e l l o w o r l d))


'((h 1) (e 1) (l 3) (l 2) (o 2) (w 1) (o 1) (r 1) (l 1) (d 1))

#### (c) Convert a trinary string represented as a list of 0s, 1s, and 2s into a (decimal) number:

```
trinary-to-decimal '(1 0 2 1 0)) ; ==> 102
```

In [60]:
(define (trinary-to-decimal trinary-list)
  (foldl (lambda (digit acc)
           (+ (* acc 3) digit))
         0
         trinary-list))

(trinary-to-decimal '(1 0 2 1 0))


102

#### (d) Return the 3rd to last symbol in a list (you may assume it has enough symbols):

```
(third-to-last '(1 1 1 0 1 1)) ; ==> 0
(third-to-last '(s y m b o l)) ; ==> 'b
```

In [81]:
(define (third-to-last lst)
  (car (foldl (lambda (sym acc)
                (list (cadr acc) (caddr acc) sym))
              (list #f #f #f)
              lst)))

(display (third-to-last '(1 1 1 0 1 1)))
(newline)
(display (third-to-last '(s y m b o l)))

0
b

#### (e) Decrement a binary number. Decrementing zero should produce zero:

```
(decrement '(1 0 1 1 0)) ; ==> '(1 0 1 0 1)
(decrement '(1 0 0 0 0)) ; ==> '(1 1 1 1)
(decrement '(0)) ; ==> '(0)
```

In [93]:
(define (decrement binary)
  (let* ([reversed (reverse binary)]
         [decremented (foldl 
                       (lambda (bit acc)
                         (let ([carry (car acc)]
                              [result (cdr acc)])
                           (cond
                             [(and (= bit 1) (= carry 0)) (cons 1 (cons 0 result))]
                             [(and (= bit 0) (= carry 0)) (cons 1 (cons 1 result))]
                             [else (cons 0 (cons bit result))])))
                       (cons 1 '())
                       reversed)]
         [result (reverse (cdr decremented))]
         [stripped (dropwhile zero? result)])
    (if (null? stripped) '(0) stripped)))

(define (dropwhile pred lst)
  (foldr (lambda (x acc)
           (if (and (null? acc) (pred x))
               '()
               (cons x acc)))
         '()
         lst))

(define (zero? x) (= x 0))

(displayln (decrement '(1 0 1 1 0)))
(displayln (decrement '(1 0 0 0 0)))
(displayln (decrement '(0)))

(0 0 1 1 1)
(0 1 0 1 1)
(0)


### 2. Consider this list where each entry is a tuple of the first name, last name, gender, and age:

```racket
(define employees
  '(("John" "Malkovich" #:male . 79)
    ("Ivan" "Petrov" #:male . 18)
    ("Anna" "Petrova" #:female . 62)
    ("Ivan" "Ivanov" #:male . 43)
    ("Anna" "Karenina" #:female . 20)))
```

#### (a) Implement a predicate `of-pensionable-age?` which is true when a given employee is of pensionable age (≥ 60 for female and ≥ 65 for male employees):

```racket
(of-pensionable-age? '("John" "Malkovich" #:male . 79))
; #t
(of-pensionable-age? '("Anna" "Petrova" #:female . 62))
; #t
```

In [13]:
(define employees
  '(("John" "Malkovich" #:male 79)
    ("Ivan" "Petrov" #:male 18)
    ("Anna" "Petrova" #:female 62)
    ("Ivan" "Ivanov" #:male 43)
    ("Anna" "Karenina" #:female 20)))

(define (of-pensionable-age? employee)
  (let ([first-name (car employee)]
        [last-name (cadr employee)]
        [gender (caddr employee)]
        [age (cadddr employee)])
    (cond
      [(and (eq? gender '#:male) (>= age 65)) #t]
      [(and (eq? gender '#:female) (>= age 60)) #t]
      [else #f])))

(display(of-pensionable-age? '("John" "Malkovich" male 79)))
(newline)
(display(of-pensionable-age? '("Anna" "Petrova" female 62)))

#f
#f

#### (b) Using higher-order functions (`map`, `ormap`, `andmap`, `filter`, `foldl`) and without explicit recursion, implement a function `employees-of-pensionable-age` that computes a list of *full names of employees who are of pensionable age given a list of employee entries as input:

``` racket
(employees-of-pensionable-age employees)
; '(("John" . "Malkovich") ("Anna" . "Petrova"))
```

In [19]:
(define employees
  '(("John" "Malkovich" #:male 79)
    ("Ivan" "Petrov" #:male 18)
    ("Anna" "Petrova" #:female 62)
    ("Ivan" "Ivanov" #:male 43)
    ("Anna" "Karenina" #:female 20)))

(define (of-pensionable-age? employee)
  (let ([first-name (car employee)]
        [last-name (cadr employee)]
        [gender (caddr employee)]
        [age (cadddr employee)])
    (cond
      [(and (eq? gender '#:male) (>= age 65)) #t]
      [(and (eq? gender '#:female) (>= age 60)) #t]
      [else #f])))

(define (employees-of-pensionable-age employees)
  (map (lambda (employee)
         (cons (car employee) (cadr employee)))
       (filter of-pensionable-age? employees)))

(employees-of-pensionable-age employees)

'(("John" . "Malkovich") ("Anna" . "Petrova"))

### Consider the following two definitions of map via `foldl`:

```
(define (map1 f xs)
  ((foldl
    (lambda (x current)
      (lambda (t) (current (cons (f x) t))))
    (lambda (t) t)
    xs)
  empty))

(define (map2 f xs)
  (foldl
   cons
   empty
  (foldl
    (lambda (x current) (cons (f x) current))
    empty
    xs)))
```

Using *Substitution Model*, show that the following definitions of `map` are equivalent. It
is easy to see that both functions are equivalent on empty lists (for any function **f**):

```
(map1 f empty)
= ((foldl (...) (lambda (t) t) empty) empty) ; by definition of map1
= ((lambda (t) t) empty) ; by definition of foldl
= empty ; by definition of identity function
= (foldl cons empty empty) ; (backwards) by definition of foldl
= (foldl cons empty (foldl (...) empty empty)) ; (backwards) by definition of foldl
= (map2 f empty) ; (backwards) by definition of map2
```

**Complete the proof** for the case when the input list is not empty, for any f, x, and xs:

```
(map1 f (cons x xs))
= ... ; <- your proof as a sequence of equalities goes here
= (map2 f (cons x xs))
```

In addition to the regular Substitution Model, you can use the following equivalences:

1. for all f, z the following expressions are equivalent:

    - (foldl f z empty)

    - z

2. for all f, z, x, xs the following expressions are equivalent:

    - (foldl f z (cons x xs))

    - (foldl f (f x z) xs)

### Proof

1. `(map1 f (cons x xs)) = ((foldl (lambda (x current) (lambda (t) (current (cons (f x) t)))) (lambda (t) t) (cons x xs)) empty)`
   
    by definition of `map1`

2. It also equals `((foldl (lambda (x current) (lambda (t) (current (cons (f x) t))))
((lambda (x current) (lambda (t) (current (cons (f x) t)))) x (lambda (t) t))
xs) empty)`

   by equivalence form (b)

3. And it substitutes to `((foldl (lambda (x current) (lambda (t) (current (cons (f x) t))))
(lambda (t) ((lambda (t) t) (cons (f x) t)))
xs) empty)`

4. Finally, by simplification of identity function we obtain: `((foldl (lambda (x current) (lambda (t) (current (cons (f x) t))))
(lambda (t) (cons (f x) t))
xs) empty) `

Let's call this intermediate function `g`:

`g = (foldl (lambda (x current) (lambda (t) (current (cons (f x) t))))
(lambda (t) (cons (f x) t))
xs)`

So we have:

`(map1 f (cons x xs)) = (g empty)`

Now, let's consider `map2`:

1. `(map2 f (cons x xs)) = (foldl cons empty (foldl (lambda (x current) (cons (f x) current)) empty (cons x xs)))`

    by definition of `map2`

2. It equals `(foldl cons empty (foldl (lambda (x current) (cons (f x) current)) (cons (f x) empty) xs))`

   by equivalence (b)

Let's call this intermediate result of the inner foldl `h`:

`h = (foldl (lambda (x current) (cons (f x) current)) (cons (f x) empty) xs)`

So we have:

`(map2 f (cons x xs)) = (foldl cons empty h)`

Now, to prove that `(map1 f (cons x xs)) = (map2 f (cons x xs))` , we need to show that `(g empty) = (foldl cons empty h)`.

We can prove this by induction on the structure of `xs`:

1. Base case `(xs is empty)`:
   
    `g = (lambda (t) (cons (f x) t))`

    `(g empty) = (cons (f x) empty)`
   
    `h = (cons (f x) empty)`
    `(foldl cons empty h) = (cons (f x) empty)`
   
So the base case holds.

2. Inductive case `(xs is (cons y ys))`:

Assume the equality holds for `ys`, we need to prove it for `(cons y ys)`.

`g = (lambda (t) (current (cons (f y) t)))` where `current` is the result of folding over `ys`

`h = (cons (f y) h')` where `h'` is the result of folding over `ys`

By the induction hypothesis:

`(g empty) = (foldl cons empty h')`

Finally:

`(g empty) = (current (cons (f y) empty))`

`= (foldl cons (cons (f y) empty) h')`    by induction hypothesis

`= (foldl cons empty (cons (f y) h'))`    by definition of foldl

`= (foldl cons empty h)`

This completes the proof that `(map1 f (cons x xs)) = (map2 f (cons x xs))` for any `f`, `x`, and `xs`.